In [ ]:
import os
from openai import OpenAI, pydantic_function_tool
import rich
import requests
import json
from pydantic import BaseModel, Field
from google.colab import userdata

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI(api_key=api_key)

**Looping through tool calls in cases where they are received in multiple steps. For scenarios involving multiple tool calls, like GetWeather and SendEmail, where each tool might be called multiple times, we need to create logic that can send and receive messages from the model in multiple stages, making a final call to receive the complete response.**

The Pydantic-generated function structure is acceptable in OpenAI's Chat API, but the Responses API requires a slightly different structure.

In [ ]:
class GetWeather(BaseModel):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")

class SendEmail(BaseModel):
    to: str = Field(..., description="Email address of the recipient")
    subject: str = Field(..., description="Subject of the email")
    body: str = Field(..., description="Body of the email")

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    print(f"get_weather function called to get weather for latitude = {latitude}, longitude = {longitude}")
    print(f"And result is  = {data['current']['temperature_2m']}")
    return data['current']['temperature_2m']

def send_email(to, subject, body):
    print(f"Tool send_email Sending email to {to} with subject {subject}")
    print(f"Body: {body}")
    print(f"Email Tool call completed")
    return "Email Sent"

rich.print(pydantic_function_tool(GetWeather))
rich.print(pydantic_function_tool(SendEmail))

# creating list of tools here because it can be used in both Chat API and Responses API
tools = [pydantic_function_tool(SendEmail), pydantic_function_tool(GetWeather)]

{
    'type': 'function',
    'function': {
        'name': 'GetWeather',
        'strict': True,
        'parameters': {
            'properties': {
                'latitude': {'description': 'Latitude of the location', 'title': 'Latitude', 'type': 'number'},
                'longitude': {'description': 'Longitude of the location', 'title': 'Longitude', 'type': 'number'}
            },
            'required': ['latitude', 'longitude'],
            'title': 'GetWeather',
            'type': 'object',
            'additionalProperties': False
        }
    }
}

{
    'type': 'function',
    'function': {
        'name': 'SendEmail',
        'strict': True,
        'parameters': {
            'properties': {
                'to': {'description': 'Email address of the recipient', 'title': 'To', 'type': 'string'},
                'subject': {'description': 'Subject of the email', 'title': 'Subject', 'type': 'string'},
                'body': {'description': 'Body of the email', 'title': 'Body', 'type': 'string'}
            },
            'required': ['to', 'subject', 'body'],
            'title': 'SendEmail',
            'type': 'object',
            'additionalProperties': False
        }
    }
}

# Chat Completion API

https://platform.openai.com/docs/guides/function-calling?api-mode=chat

Defining a function that will receive list of messages and call the Chat API

In [ ]:
# Function to call OpenAI's Chat API
def call_chat_api(messages):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )
    return response

Defining a function `handle_tool_call` in which we will loop through the tool_calls, invoke the corresponding functions, and update the response. Plus, we will append a new message for each function call to promptMessages.

In [ ]:
def handle_tool_call(promptMessages, responseMessage):
    for tool_call in responseMessage.tool_calls:
        name = tool_call.function.name
        args = json.loads(tool_call.function.arguments)
        result = None
        if name == "SendEmail":
            result = send_email(**args)
        elif name == "GetWeather":
            result = get_weather(**args)
        new_message = {
            "role": "tool",
            "content": str(result),
            "tool_call_id": tool_call.id
        }
        promptMessages.append(new_message)

The `handle_chat_api()` function handles communication with a chat API in a loop, processing user prompts, handling tool calls, and iterating until a final response is received.

In [ ]:
def handle_chat_api(prompt):
    messages=[
        {"role": "developer", "content": "You are a helpful assistant, you can send email about weather in a city. Email should be courteous and professional."},
        {"role": "user", "content": prompt}
    ]

    response = call_chat_api(messages)
    counter = 0 # counter to keep track of the number times we receive tool calls from model

    while True:
        print("Finish Reason = ",response.choices[0].finish_reason)
        print("Tool Calls Length = ",len(response.choices[0].message.tool_calls or []))
        print("Tool Calls = ",response.choices[0].message.tool_calls)
        print("--")
        if(response.choices[0].finish_reason == "stop"):
            print("-------")
            print("Number of times Tool calls received from model are = ",counter)
            print()
            print(response.choices[0].message.content)
            print("-------")
            break
        elif response.choices[0].finish_reason == "tool_calls":
            counter += 1
            messages.append(response.choices[0].message)
            handle_tool_call(messages, response.choices[0].message)

            # Uncomment below line to see the messages
            # rich.print(messages)

            # calling chat api again with updated messages and storing in same response variable
            response = call_chat_api(messages)
        print()


Call `handle_chat_api` function with different inputs

In [ ]:
# Check result of each prompt

# handle_chat_api("Send an email to shan@gmail.com and hello@gmail.com about weather in Karachi")
# handle_chat_api("Send an email about weather in Karachi to first@gmail.com and second@gmail.com")
handle_chat_api("send an email about weather in Karachi and Lahore to both shan@gmail.com and hello@gmail.com")
# handle_chat_api("send an email about weather in Karachi and Lahore to shan@gmail.com and hello@gmail.com")

Finish Reason =  tool_calls
Tool Calls Length =  2
Tool Calls =  [ChatCompletionMessageToolCall(id='call_xWIE6IRY2kUVM1gJWWE4tTOL', function=Function(arguments='{"latitude": 24.8607, "longitude": 67.0011}', name='GetWeather'), type='function'), ChatCompletionMessageToolCall(id='call_hUYWNhNnb1ec57fR9KlmAkTf', function=Function(arguments='{"latitude": 31.5497, "longitude": 74.3436}', name='GetWeather'), type='function')]
--
get_weather function called to get weather for latitude = 24.8607, longitude = 67.0011
And result is  = 25.4
get_weather function called to get weather for latitude = 31.5497, longitude = 74.3436
And result is  = 22.4

Finish Reason =  tool_calls
Tool Calls Length =  1
Tool Calls =  [ChatCompletionMessageToolCall(id='call_SNROPZ3KSUWRSmL7rCAZxrNe', function=Function(arguments='{"to":"shan@gmail.com","subject":"Current Weather Update for Karachi and Lahore","body":"Dear Shan,\\n\\nI hope this message finds you well. \\n\\nI wanted to share the current weather updates 

# Responses API

https://platform.openai.com/docs/guides/function-calling?api-mode=responses

To use pydantic-generated function, we need to use `openai.responses.parse()` function call

https://github.com/openai/openai-python/blob/main/examples/responses/structured_outputs_tools.py

### Using old way of sending history messages in every call

Example of new way of handling messages is in separate notebook

Defining a function that will receive list of messages and call the Responses API

In [ ]:
# Function to call OpenAI's Chat API
def call_response_api(messages):
    response = openai.responses.parse(
        model=MODEL,
        input=messages,
        tools = tools
    )
    return response

Defining a function `handle_tool_call_responses` in which we will loop through the tool_calls, invoke the corresponding functions, and update the response. Plus, we will append a new message for each function call to promptMessages.

In [ ]:
def handle_tool_call_responses(promptMessages, responseFromAPI):
    isFunctionCall = False
    for tool_call in responseFromAPI.output:
        if tool_call.type == "function_call": # check if the output's type is function_call
            # As we don't have finish_reason property in Responses API that indicate tool_calls or stop
            # so we will use isFunctionCall flag to keep track it function call is received
            isFunctionCall = True
            name = tool_call.name
            arguments = json.loads(tool_call.arguments)
            if name == "SendEmail":
                result = send_email(**arguments)
            elif name == "GetWeather":
                result = get_weather(**arguments)
            new_message = {
                "type": "function_call_output",
                "call_id": tool_call.call_id,
                "output": str(result)
            }
            # In case of Response API we need to append individual output messages to messages list
            # Therefore, we append tool_call and new_message to promptMessages list one by one
            # Plus we need to remove the parsed_arguments from tool_call object
            del tool_call.parsed_arguments
            promptMessages.append(tool_call)
            promptMessages.append(new_message)
    return isFunctionCall

The `handle_response_api()` function handles communication with a Responses API in a loop, processing user prompts, handling tool calls, and iterating until a final response is received.

In [ ]:
def handle_response_api(prompt):
    messages=[
        {"role": "developer", "content": "You are a helpful assistant, you can send email about weather in a city. Email should be courteous and professional."},
        {"role": "user", "content": prompt}
    ]

    response = call_response_api(messages)
    counter = 0 # counter to keep track of the number times we receive tool calls from model

    while True:
        print()
        print("Iteration Started")
        print("Response Status = ",response.status)
        # rich.print("response.output = ",response.output)
        # We don't have finish_reason in response from Response API, and status always be "completed"
        isFunctionCalled = handle_tool_call_responses(messages, response)
        if isFunctionCalled:
            counter += 1
            # Uncomment below line to see the messages
            # rich.print(messages)
            response = call_response_api(messages)
        else:
            print("-------")
            print("Number of times Tool calls received from model are = ",counter)
            print()
            print(response.output_text)
            print("-------")
            break

Call `handle_response_api` function with different inputs

In [ ]:
# Check result of each prompt
handle_response_api("Send an email to shan@gmail.com and hello@gmail.com about weather in Karachi")
# handle_response_api("Send an email about weather in Karachi to first@gmail.com and second@gmail.com") # This prompt is giving error somehow
# handle_response_api("send an email about weather in Karachi and Lahore to both shan@gmail.com and hello@gmail.com")
# handle_response_api("send an email about weather in Karachi and Lahore to shan@gmail.com and hello@gmail.com")


Iteration Started
Response Status =  completed
get_weather function called to get weather for latitude = 24.8607, longitude = 67.0011
And result is  = 25.4

Iteration Started
Response Status =  completed
Tool send_email Sending email to shan@gmail.com with subject Weather Update for Karachi
Body: Dear Shan,

I hope this email finds you well. 

I wanted to provide you with a brief weather update for Karachi. The current temperature is approximately 25.4°C. 

Please let me know if you need any further information.

Best regards,

Your Assistant
Email Tool call completed

Iteration Started
Response Status =  completed
Tool send_email Sending email to hello@gmail.com with subject Weather Update for Karachi
Body: Dear [Recipient's Name],

I hope this email finds you well. 

I wanted to provide you with a brief weather update for Karachi. The current temperature is approximately 25.4°C. 

Please let me know if you need any further information.

Best regards,

Your Assistant
Email Tool call 